In [21]:
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
from pytensor.scan import scan
import arviz as az

In [22]:
import pandas as pd
import glob
import os
import re

# Folder where your files are
folder = "../data/processed"

# Pattern to match 'hmm_<asset>' at the start of the filename
pattern = re.compile(r'hmm_([A-Za-z0-9]+)')

# Find all CSV files starting with "hmm_"
files = glob.glob(os.path.join(folder, "hmm_*.csv"))

returns = []

for file in files:
    filename = os.path.basename(file)
    match = pattern.match(filename)
    if match:
        asset = match.group(1)
        df = pd.read_csv(
            file,
            index_col=0,       # use first column as index
            parse_dates=True   # parse index as datetime if possible
        )
        
        returns.append(df[f"{asset}_ret"])

portfolio_returns = (
    pd.concat(returns, axis=1)
)

In [24]:
num_assets = len(portfolio_returns.columns)
num_regimes = 2
T = len(portfolio_returns)

In [25]:
import pymc as pm
import pytensor.tensor as pt
import numpy as np

# ----------------------------
# Inputs
# ----------------------------
# returns_np: (T, N) array of daily returns
T, N = returns_np.shape
num_regimes = 2  # e.g., low-volatility, high-volatility

# ----------------------------
# HMM Model
# ----------------------------
with pm.Model() as hmm_model:
    
    # --- Regime transition matrix ---
    P = pm.Dirichlet("P", a=np.ones((num_regimes, num_regimes)), shape=(num_regimes, num_regimes))
    
    # --- Initial regime probabilities ---
    pi0 = pm.Dirichlet("pi0", a=np.ones(num_regimes))
    
    # --- Regime-specific means and covariances ---
    mu = pm.Normal("mu", mu=0, sigma=0.01, shape=(num_regimes, N))
    
    sd_dist = pm.Exponential.dist(1.0, shape=N)
    chol, corr, stds = pm.LKJCholeskyCov("chol_cov", n=N, eta=2, sd_dist=sd_dist)
    
    # Compute full covariance matrices for each regime
    sigma = pm.Deterministic("sigma", pt.stack([pt.dot(chol, chol.T) for _ in range(num_regimes)]))
    
    # --- Hidden regimes z_t ---
    # Categorical latent variable with Markov dependency
    z = pm.CategoricalHMM("z", P=P, pi0=pi0, shape=T)
    
    # --- Observation model ---
    obs = pm.MvNormal(
        "obs",
        mu=mu[z],
        chol=chol,  # use cholesky for multivariate normal
        observed=returns_np
    )
    
    # --- Sample ---
    idata = pm.sample(chains=4, draws=2000, tune=2000, target_accept=0.95)


AttributeError: module 'pymc' has no attribute 'CategoricalHMM'

In [ ]:
idata.to_netcdf("bayes_hmm_portfolio.nc")

In [ ]:
break

In [ ]:
import numpy as np
from scipy.optimize import minimize

# ----------------------------
# Inputs
# ----------------------------
# returns_np: (T, N) array of daily returns
# mu_t: (T, N) array of expected returns at each time
# sigma_t: (T, N, N) array of full covariance matrices at each time

T, N = returns_np.shape
weights_t = np.zeros((T, N))
port_ret_t = np.zeros(T)

# ----------------------------
# Sharpe-maximizing function
# ----------------------------
def sharpe_weights(mu, sigma):
    # objective: negative Sharpe
    def neg_sharpe(w):
        port_mean = np.dot(w, mu)
        port_var = np.dot(w, np.dot(sigma, w))
        return -port_mean / np.sqrt(port_var)
    
    # constraints: sum(weights) = 1, weights >= 0
    cons = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1) for _ in range(len(mu))]
    
    # initial guess: equal weight
    w0 = np.ones(len(mu)) / len(mu)
    res = minimize(neg_sharpe, w0, bounds=bounds, constraints=cons)
    
    return res.x

# ----------------------------
# Compute time-varying portfolio
# ----------------------------
for t in range(T):
    # skip NaNs
    if np.any(np.isnan(mu_t[t])) or np.any(np.isnan(sigma_t[t])):
        weights_t[t] = np.nan
        port_ret_t[t] = np.nan
        continue
    
    w_t = sharpe_weights(mu_t[t], sigma_t[t])
    weights_t[t] = w_t
    port_ret_t[t] = np.dot(returns_np[t], w_t)

# --


In [ ]:
import numpy as np
import pandas as pd

# ----------------------------
# Compute portfolio returns
# ----------------------------
portfolio_returns = np.nansum(returns_np * weights_t, axis=1)
valid_idx = ~np.isnan(portfolio_returns)
portfolio_returns_clean = portfolio_returns[valid_idx]

# Equal-weight benchmark
benchmark_returns = np.mean(returns_np[valid_idx, :], axis=1)
benchmark_returns_clean = benchmark_returns[~np.isnan(benchmark_returns)]

# ----------------------------
# Function to compute metrics
# ----------------------------
def compute_metrics(returns):
    mean = returns.mean()
    vol = returns.std()
    sharpe = mean / vol * np.sqrt(252)
    cum_returns = np.cumprod(1 + returns) - 1
    max_drawdown = (cum_returns - np.maximum.accumulate(cum_returns)).min()
    return mean, vol, sharpe, max_drawdown

# ----------------------------
# Metrics table
# ----------------------------
metrics = []
metrics.append(["HMM Portfolio", *compute_metrics(portfolio_returns_clean)])
metrics.append(["Equal-Weight Benchmark", *compute_metrics(benchmark_returns_clean)])

metrics_df = pd.DataFrame(metrics, columns=["Strategy", "Mean Return", "Volatility", "Annualized Sharpe", "Max Drawdown"])
print(metrics_df)

# ----------------------------
# Weight statistics table
# ----------------------------
weights_df = pd.DataFrame(weights_t, columns=[f"Asset {i}" for i in range(weights_t.shape[1])])
weight_stats = pd.DataFrame({
    "Mean Weight": weights_df.mean(),
    "Min Weight": weights_df.min(),
    "Max Weight": weights_df.max()
})
print("\nWeight Statistics:")
print(weight_stats)

# ----------------------------
# Rolling Sharpe summary
# ----------------------------
rolling_window = 20
rolling_mean = pd.Series(portfolio_returns_clean).rolling(rolling_window).mean()
rolling_std = pd.Series(portfolio_returns_clean).rolling(rolling_window).std()
rolling_sharpe = rolling_mean / rolling_std * np.sqrt(252)

rolling_sharpe_stats = pd.DataFrame({
    "Rolling Sharpe (20d)": [rolling_sharpe.min(), rolling_sharpe.mean(), rolling_sharpe.max()]
}, index=["Min", "Mean", "Max"])
print("\nRolling Sharpe (20-day) Statistics:")
print(rolling_sharpe_stats)


In [ ]:
import numpy as np
import pandas as pd

# Assume returns_np, weights_t, benchmark_idx as before

# ----------------------------
# 1. Portfolio daily returns
# ----------------------------
portfolio_returns_2 = np.sum(returns_np * weights_t, axis=1)
valid_idx = ~np.isnan(portfolio_returns)
portfolio_returns_clean = portfolio_returns_2[valid_idx]

benchmark_returns = np.mean(returns_np, axis=1)
benchmark_returns_clean = benchmark_returns[~np.isnan(benchmark_returns)]

# ----------------------------
# 2. Key metrics table
# ----------------------------
def compute_metrics(returns):
    mean = returns.mean()
    std = returns.std()
    sharpe = mean / std * np.sqrt(252)
    cum_returns = np.cumprod(1 + returns) - 1
    max_drawdown = (cum_returns - np.maximum.accumulate(cum_returns)).min()
    return mean, std, sharpe, max_drawdown

metrics = []
metrics.append(["HMM Portfolio", *compute_metrics(portfolio_returns_clean)])
metrics.append([f"Benchmark (Asset {benchmark_idx})", *compute_metrics(benchmark_returns_clean)])

metrics_df = pd.DataFrame(metrics, columns=["Strategy", "Mean Return", "Volatility", "Annualized Sharpe", "Max Drawdown"])
print(metrics_df)

# ----------------------------
# 3. Weight statistics table
# ----------------------------
weights_df = pd.DataFrame(weights_t, columns=[f"Asset {i}" for i in range(weights_t.shape[1])])
weight_stats = pd.DataFrame({
    "Mean Weight": weights_df.mean(),
    "Min Weight": weights_df.min(),
    "Max Weight": weights_df.max()
})
print("\nWeight Statistics:")
print(weight_stats)

# ----------------------------
# 4. Optional: Rolling Sharpe table
# ----------------------------
rolling_window = 20
rolling_mean = pd.Series(portfolio_returns_clean).rolling(rolling_window).mean()
rolling_std = pd.Series(portfolio_returns_clean).rolling(rolling_window).std()
rolling_sharpe = rolling_mean / rolling_std * np.sqrt(252)

rolling_sharpe_stats = pd.DataFrame({
    "Rolling Sharpe (20d)": [rolling_sharpe.min(), rolling_sharpe.mean(), rolling_sharpe.max()]
}, index=["Min", "Mean", "Max"])

print("\nRolling Sharpe (20-day) Statistics:")
print(rolling_sharpe_stats)


In [ ]:
import arviz as az
idata = az.from_netcdf("bayes_hmm_portfolio.nc")

In [ ]:
z_samples = idata.posterior["z"].values  # (chains, draws, T) e.g., (2, 500, 2496)
pi_t_posterior = np.zeros((T, num_regimes))  # (2496, 2)

for k in range(num_regimes):
    pi_t_posterior[:, k] = np.mean(z_samples == k, axis=(0, 1)) 
    
mu_post = idata.posterior["mu"].mean(('chain', 'draw')).values  # (regimes, assets)
sigma_post = idata.posterior["sigma"].mean(('chain', 'draw')).values ** 2  # variances

In [ ]:
def sharpe_weights(mu_t, Sigma_t, leverage=1.0):
    """Analytical Sharpe-maximizing weights (full or diagonal Sigma_t)."""
    try:
        Sigma_inv = np.linalg.pinv(Sigma_t)
        w = Sigma_inv @ mu_t
        w = w / np.sum(np.abs(w)) * leverage
        return w
    except Exception:
        return np.zeros(len(mu_t))

weights_t = np.zeros((T, num_assets))
port_sharpe_t = np.zeros(T)

for t in range(T):
    pi_t = pi_t_posterior[t, :]                    # (num_regimes,)
    
    # Mixture mean per asset
    mu_t = np.sum(pi_t[:, None] * mu_post, axis=0)  # (N,)
    
    # Mixture *variance* per asset (diagonal)
    var_t_diag = np.sum(pi_t[:, None] * sigma_post**2, axis=0)  # (N,)
    
    # Build diagonal covariance for the optimizer
    Sigma_t = np.diag(var_t_diag)                  # (N, N)

    # Sharpe-max weights
    w_t = sharpe_weights(mu_t, Sigma_t)
    weights_t[t] = w_t

    # Realized Sharpe using diagonal variance
    port_mu_t = w_t @ mu_t
    port_var_t = np.sum((w_t**2) * var_t_diag)
    port_sharpe_t[t] = port_mu_t / np.sqrt(port_var_t)

In [ ]:
print("\n=== TIME-VARYING SHARPE PORTFOLIOS ===")
print(f"Current regime probs: [{pi_t_posterior[-1,0]:.1%}, {pi_t_posterior[-1,1]:.1%}]")
print("\nCurrent weights:", weights_t[-1])
print(f"Current Sharpe: {port_sharpe_t[-1]:.3f}")

print("\nWeight evolution (first → last):")
for i, name in enumerate(portfolio_returns.columns):
    print(f"{name}: {weights_t[0, i]:.3f} → {weights_t[-1, i]:.3f}")

print(f"\nSharpe evolution: {port_sharpe_t[0]:.2f} → {port_sharpe_t[-1]:.2f}")
print(f"Max Sharpe: {port_sharpe_t.max():.2f}")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,3))
plt.plot(port_sharpe_t, label="Sharpe over time")
plt.legend()
plt.title("Time-Varying Sharpe")
plt.show()

plt.figure(figsize=(10,4))
for i, a in enumerate(portfolio_returns.columns):
    plt.plot(weights_t[:,i], label=a)
plt.legend()
plt.title("Bayesian Regime-Conditional Weights")
plt.show()


In [ ]:
np.save("../data/processed/bayesian_weights.npy", weights_t)
np.save("../data/processed/bayesian_regime_probs.npy", pi_t)